# Just Numpy

Let's start with a plausible problem. We have a dataset of all daily temperatures measured at Newark since 1893 and we want to analyze it. First, let's try that with a Python list.

In [1]:
temperatures = []
with open("data/newark-temperature-avg.txt") as file:
    for line in file.readlines():
        temperatures.append(float(line))

len(temperatures), temperatures[:10], temperatures[-10:]

(42019,
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
 [43.0, 47.0, 49.0, 52.0, 48.0, 52.0, 62.0, 68.0, 59.0, 47.0])

Much of the record is missing, as we can see by counting NaNs:

In [2]:
import math
numbad = 0
for x in temperatures:
    if math.isnan(x):
        numbad += 1

numbad / len(temperatures)

0.887717461148528

We have a more complete dataset of daily minimum and maximum temperatures. It's not as accurate, but we can impute the missing averages by averaging the minimum and maximum.

In [3]:
min_temperatures = []
with open("data/newark-temperature-min.txt") as file:
    for line in file.readlines():
        min_temperatures.append(float(line))

max_temperatures = []
with open("data/newark-temperature-max.txt") as file:
    for line in file.readlines():
        max_temperatures.append(float(line))

(len(min_temperatures), min_temperatures[:10], min_temperatures[-10:],
 len(max_temperatures), max_temperatures[:10], max_temperatures[-10:])

(42019,
 [26.0, 34.0, 17.0, 13.0, 17.0, 13.0, 12.0, 15.0, 11.0, 4.0],
 [36.0, 45.0, 45.0, 44.0, 39.0, 37.0, 52.0, 65.0, 46.0, nan],
 42019,
 [52.0, 43.0, 32.0, 23.0, 27.0, 30.0, 28.0, 28.0, 32.0, 32.0],
 [50.0, 50.0, 54.0, 59.0, 58.0, 68.0, 73.0, 73.0, 67.0, nan])

While we fill in the missing values, let's also measure how long it takes.

In [4]:
%%timeit

imputed_temperatures = []
for average, minimum, maximum in zip(temperatures, min_temperatures, max_temperatures):
    if math.isnan(average):
        imputed_temperatures.append(0.5 * (minimum + maximum))
    else:
        imputed_temperatures.append(average)

37 ms ± 366 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Now let's do the same thing in Numpy, again measuring the time.

In [5]:
import numpy

temperatures = numpy.array(temperatures)
min_temperatures = numpy.array(min_temperatures)
max_temperatures = numpy.array(max_temperatures)

In [6]:
%%timeit

missing = numpy.isnan(temperatures)
imputed_temperatures = numpy.empty(len(temperatures), dtype=numpy.float64)
imputed_temperatures[missing] = 0.5 * (min_temperatures[missing] + max_temperatures[missing])
imputed_temperatures[~missing] = temperatures[~missing]

1.27 ms ± 34.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Or just

In [7]:
%%timeit

imputed_temperatures = numpy.where(
    # condition                # if true                                    # if false
    numpy.isnan(temperatures), 0.5 * (min_temperatures + max_temperatures), temperatures)

730 µs ± 61.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We see that Numpy can be much faster than Python loops, in this case a factor of 10 or 20, but I have seen as much as 1000. (It depends on the application.) The way you tell it what to do is also very different, which may be good or bad. It may read more naturally, maybe not.

One thing we saw was a preoccupation on data types, unusual for Python.

In [8]:
numpy.zeros(5, dtype=numpy.float64)

array([0., 0., 0., 0., 0.])

In [9]:
numpy.zeros(5, dtype=numpy.int32)

array([0, 0, 0, 0, 0], dtype=int32)

In [10]:
numpy.zeros(5, dtype=numpy.bool)

array([False, False, False, False, False])

In [11]:
numpy.zeros(5, dtype="S3")

array([b'', b'', b'', b'', b''], dtype='|S3')

This is where a large part of Numpy's speed comes from. When Python churns, a lot of that time is spent checking and re-checking data types, which in a compiled language like C++ were checked once and for all in the compilation step.

Numpy is a suite of compiled functions applied to data with predetermined types. When you're using Numpy properly, you'll have very few `for` loops and `if` statements in your code: the Python code acts as a high-level director, while Numpy does its looping in compiled code.

The Numpy library consists mainly of one class, `numpy.ndarray`, and operations on it. This is an n-dimensional array of contiguous data. Some operations change that data or make new arrays, but many operations merely change our interpretation of the data. The latter are the fastest.

In [12]:
array = numpy.arange(24, dtype=numpy.float64)    # 64-bit floating point numbers
array

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.])

In [13]:
array.view(numpy.int64)

array([                  0, 4607182418800017408, 4611686018427387904,
       4613937818241073152, 4616189618054758400, 4617315517961601024,
       4618441417868443648, 4619567317775286272, 4620693217682128896,
       4621256167635550208, 4621819117588971520, 4622382067542392832,
       4622945017495814144, 4623507967449235456, 4624070917402656768,
       4624633867356078080, 4625196817309499392, 4625478292286210048,
       4625759767262920704, 4626041242239631360, 4626322717216342016,
       4626604192193052672, 4626885667169763328, 4627167142146473984])

In [72]:
array.view("S8")

array([[b'', b'\x9a\x99\x99\x99\x99\x99\xf1?',
        b'\x9a\x99\x99\x99\x99\x99\x01@', b'ffffff\n@'],
       [b'', b'333333$@', b'3333334@', b'\xcd\xcc\xcc\xcc\xccL>@'],
       [b'', b'fffff\x06Y@', b'fffff\x06i@',
        b'\xcd\xcc\xcc\xcc\xcc\xc4r@']], dtype='|S8')

In [73]:
array.tostring()

b'\x00\x00\x00\x00\x00\x00\x00\x00\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@ffffff\n@\x00\x00\x00\x00\x00\x00\x00\x00333333$@3333334@\xcd\xcc\xcc\xcc\xccL>@\x00\x00\x00\x00\x00\x00\x00\x00fffff\x06Y@fffff\x06i@\xcd\xcc\xcc\xcc\xcc\xc4r@'

In [75]:
numpy.array([b"one", b"two", b"three"]).view(numpy.uint8)

array([111, 110, 101,   0,   0, 116, 119, 111,   0,   0, 116, 104, 114,
       101, 101], dtype=uint8)

In [77]:
list(b"one\x00\x00two\x00\x00three")

[111, 110, 101, 0, 0, 116, 119, 111, 0, 0, 116, 104, 114, 101, 101]

In [14]:
array.reshape(6, 4)

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.],
       [12., 13., 14., 15.],
       [16., 17., 18., 19.],
       [20., 21., 22., 23.]])

In [15]:
array.reshape(6, 4, order="f")    # Fortran order vs C order: how a 1D sequence covers an nD block

array([[ 0.,  6., 12., 18.],
       [ 1.,  7., 13., 19.],
       [ 2.,  8., 14., 20.],
       [ 3.,  9., 15., 21.],
       [ 4., 10., 16., 22.],
       [ 5., 11., 17., 23.]])

This interpretation has only two parameters:

   * `dtype` (data type, including endianness): how bytes are represented as numbers
   * `shape` and `order`: how those numbers are arranged in an n-dimensional grid

Mistakes in interpretation are usually not subtle, so just be sure to _look_ at your data.

Numpy arrays can be used in mathematical formulae, but instead of computing one value, they compute a whole array of values, element by element.

In [16]:
a = numpy.arange(10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [17]:
a + 100

array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109])

In [18]:
b = numpy.array([0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])

In [19]:
a + b

array([ 0. ,  2.1,  4.2,  6.3,  8.4, 10.5, 12.6, 14.7, 16.8, 18.9])

In [20]:
a**2

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

Generally, you can imagine a table of data to compute: the columns represent meaningful quantities (often named) while the rows represent anonymous instances.

In [278]:
a = numpy.random.uniform(5, 10, 10000)
b = numpy.random.uniform(10, 20, 10000)
c = numpy.random.uniform(-0.1, 0.1, 10000)
len(a)

10000

A conventional Python approach would be to compute the formula on each instance, one after another.

In [279]:
roots1 = []
for ai, bi, ci in zip(a, b, c):
    roots1.append((-bi + math.sqrt(bi**2 - 4*ai*ci)) / (2*ai))

The Numpy approach computes each step of the formula on all instances before moving on to the next step.

In [280]:
roots2 = (-b + numpy.sqrt(b**2 - 4*a*c)) / (2*a)

It is equivalent to:

In [287]:
# (-b + numpy.sqrt(b**2 - 4*a*c)) / (2*a)
tmp1 = numpy.negative(b)            # -b
tmp2 = numpy.square(b)              # b**2
tmp3 = numpy.multiply(4, a)         # 4*a
tmp4 = numpy.multiply(tmp3, c)      # tmp3*c
tmp5 = numpy.subtract(tmp2, tmp4)   # tmp2 - tmp4
tmp6 = numpy.sqrt(tmp5)             # sqrt(tmp5)
tmp7 = numpy.add(tmp1, tmp6)        # tmp1 + tmp6
tmp8 = numpy.multiply(2, a)         # 2*a
roots3 = numpy.divide(tmp7, tmp8)   # tmp7 / tmp8

One strange (but useful!) consequence of this rule that mathematical operations are applied elementwise is that it even applies to comparisons. Suppose we want to verify that the `roots1` computed in the Python loop match the `roots2` and `roots3` computed by Numpy.

In [288]:
roots1 == roots2

array([ True,  True,  True, ...,  True,  True,  True])

In [289]:
roots2 == roots3

array([ True,  True,  True, ...,  True,  True,  True])

When you want to check that _all_ of the elements are equal, I'd use `.all()`.

In [290]:
(roots2 == roots3).all()

True

In [291]:
(roots1 == roots2).all()

False

Why is that? Didn't we just see that `roots1 == roots2` is `True, True, True, ...`?

In [292]:
(roots1 == roots2).any()

True

In [293]:
(roots1 == roots2).sum(), len(roots1)

(9852, 10000)

Which ones fail?

In [294]:
numpy.nonzero(roots1 != roots2)

(array([  23,   61,  168,  230,  302,  307,  312,  336,  603,  608,  664,
         724,  729,  731,  808,  812,  868,  964,  966, 1130, 1191, 1193,
        1287, 1315, 1333, 1338, 1407, 1413, 1544, 1545, 1548, 1572, 1610,
        1717, 1742, 1764, 1871, 2066, 2098, 2217, 2255, 2382, 2398, 2430,
        2533, 2614, 2678, 2724, 2739, 2773, 2856, 2881, 2929, 3003, 3117,
        3129, 3133, 3208, 3246, 3251, 3270, 3272, 3303, 3652, 3669, 3676,
        3716, 3723, 3815, 3885, 3908, 3915, 3945, 3977, 4143, 4175, 4358,
        4472, 4487, 4595, 4995, 5056, 5087, 5141, 5201, 5262, 5291, 5359,
        5555, 5556, 5621, 5805, 6130, 6195, 6287, 6501, 6509, 6551, 6625,
        6649, 6735, 6777, 6945, 7125, 7192, 7231, 7273, 7506, 7556, 7724,
        7754, 7765, 7789, 7819, 7869, 7890, 7917, 8019, 8079, 8226, 8365,
        8393, 8484, 8578, 8590, 8618, 8628, 8676, 8808, 8930, 9060, 9063,
        9105, 9156, 9320, 9345, 9365, 9418, 9570, 9576, 9794, 9818, 9824,
        9938, 9949, 9957, 9978, 9983])

In [295]:
roots1[35], roots2[35]

(0.005157197839900364, 0.005157197839900364)

In [296]:
roots1[35] - roots2[35]

0.0

Numpy uses different routines to do its calculations, so results might not be exactly the same. We don't care about last-digit differences, so we set a tolerance.

In [297]:
(abs(roots1 - roots2) < 1e-15).all()

True

The upshot of this is that you can perform a calculation with the same expression on Numpy arrays as on Python scalars, as long as it's being applied to a table of numbers (i.e. arrays of all the same length).

Now let's get into some fancier gymnastics.

Python has a wonderfully consistent syntax for _slicing_ lists (or tuples or whatever):

In [35]:
alist = [0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9]

In [36]:
alist[4]

4.4

In [37]:
alist[4:]

[4.4, 5.5, 6.6, 7.7, 8.8, 9.9]

In [38]:
alist[:7]

[0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6]

In [39]:
alist[-1]

9.9

In [40]:
alist[:-3]

[0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6]

In [41]:
alist[3:8]

[3.3, 4.4, 5.5, 6.6, 7.7]

In [42]:
alist[3:8:2]

[3.3, 5.5, 7.7]

In [43]:
alist[::2]

[0.0, 2.2, 4.4, 6.6, 8.8]

In [44]:
alist[::-2]

[9.9, 7.7, 5.5, 3.3, 1.1]

In [45]:
alist[5::-1]

[5.5, 4.4, 3.3, 2.2, 1.1, 0.0]

Numpy arrays share this syntax— everything that works for a list works for an array— but they extend it considerably.

In [46]:
array = numpy.array([[0.0, 1.1, 2.2, 3.3], [0, 10.1, 20.2, 30.3], [0, 100.1, 200.2, 300.3]])
array

array([[  0. ,   1.1,   2.2,   3.3],
       [  0. ,  10.1,  20.2,  30.3],
       [  0. , 100.1, 200.2, 300.3]])

In [47]:
array[1]

array([ 0. , 10.1, 20.2, 30.3])

In [48]:
array[:, 1]

array([  1.1,  10.1, 100.1])

In [49]:
array[1:, 2:]

array([[ 20.2,  30.3],
       [200.2, 300.3]])

In [50]:
array[::2, 1::2]

array([[  1.1,   3.3],
       [100.1, 300.3]])

In [51]:
array[[False, True, True]]

array([[  0. ,  10.1,  20.2,  30.3],
       [  0. , 100.1, 200.2, 300.3]])

In [52]:
array[[2, 1, 0]]

array([[  0. , 100.1, 200.2, 300.3],
       [  0. ,  10.1,  20.2,  30.3],
       [  0. ,   1.1,   2.2,   3.3]])

In [53]:
array[[2, 1, 1, 1, 1, 0, 0, 2]]

array([[  0. , 100.1, 200.2, 300.3],
       [  0. ,  10.1,  20.2,  30.3],
       [  0. ,  10.1,  20.2,  30.3],
       [  0. ,  10.1,  20.2,  30.3],
       [  0. ,  10.1,  20.2,  30.3],
       [  0. ,   1.1,   2.2,   3.3],
       [  0. ,   1.1,   2.2,   3.3],
       [  0. , 100.1, 200.2, 300.3]])

What could this possibly be useful for?

**Masking:**

In [54]:
a = numpy.array([0.0, 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])
b = numpy.array([0, 100, 200, 300, 400, 500, 600, 700, 800, 900])

In [55]:
b > 400

array([False, False, False, False, False,  True,  True,  True,  True,
        True])

In [56]:
a[b > 400]

array([5.5, 6.6, 7.7, 8.8, 9.9])

**Sorting and maximizing:**

In [57]:
a = numpy.random.normal(0, 5, 50)
b = abs(a)

In [58]:
numpy.argsort(a)

array([36, 25, 35, 30, 11, 43, 22,  9, 34,  1,  0,  2, 48, 42, 19, 47, 20,
       10, 29, 12, 44,  5,  7, 14, 45, 17, 39, 40, 37, 23, 26, 24,  8, 32,
       15, 27,  6,  3, 46, 21, 13, 41, 49, 31, 33, 38,  4, 28, 18, 16])

In [59]:
a[numpy.argsort(a)]

array([-10.05908289,  -9.01523872,  -8.51352342,  -8.39957145,
        -8.1871671 ,  -8.07977793,  -7.87310254,  -7.78439273,
        -7.54418078,  -7.03775265,  -6.41885289,  -6.33309516,
        -6.23946325,  -5.91194849,  -5.27604034,  -5.20038265,
        -4.7724411 ,  -4.36433725,  -4.2250898 ,  -4.03527612,
        -3.70579841,  -3.41736677,  -3.00275851,  -2.79227831,
        -2.50836041,  -2.08350925,  -1.78546704,  -1.00568739,
        -0.87146695,  -0.45348838,   0.3563854 ,   0.4408167 ,
         0.91806124,   1.19807111,   1.47974809,   1.80446342,
         2.03988464,   2.09781916,   2.28446124,   3.08875416,
         3.5826108 ,   3.60552276,   4.60322866,   4.90690311,
         5.26081085,   5.78555993,   6.23469459,   7.10764997,
         7.90251011,   9.17869981])

In [60]:
a = numpy.meshgrid(numpy.linspace(-5, 5, 11), numpy.linspace(-5, 5, 11))[1]
a

array([[-5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5.],
       [-4., -4., -4., -4., -4., -4., -4., -4., -4., -4., -4.],
       [-3., -3., -3., -3., -3., -3., -3., -3., -3., -3., -3.],
       [-2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
       [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
       [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
       [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.]])

In [61]:
a.argmax(axis=0)

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])

In [62]:
a[a.argmax(axis=0)]

array([[5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.]])

**Dictionary encoding:**

In [63]:
text = """Four score and seven years ago our fathers brought forth on this
continent, a new nation, conceived in Liberty, and dedicated to the proposition
that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any
nation so conceived and so dedicated, can long endure. We are met on a great
battle-field of that war. We have come to dedicate a portion of that field, as
a final resting place for those who here gave their lives that that nation might
live. It is altogether fitting and proper that we should do this.

But, in a larger sense, we can not dedicate -- we can not consecrate -- we can
not hallow -- this ground. The brave men, living and dead, who struggled here,
have consecrated it, far above our poor power to add or detract. The world will
little note, nor long remember what we say here, but it can never forget what
they did here. It is for us the living, rather, to be dedicated here to the
unfinished work which they who fought here have thus far so nobly advanced. It
is rather for us to be here dedicated to the great task remaining before us --
that from these honored dead we take increased devotion to that cause for which
they gave the last full measure of devotion -- that we here highly resolve that
these dead shall not have died in vain -- that this nation, under God, shall
have a new birth of freedom -- and that government of the people, by the people,
for the people, shall not perish from the earth."""

In [64]:
words = text.replace(".", "").replace(",", "").replace("--", "").split()

In [65]:
dictionary, integers = numpy.unique(words, return_inverse=True)

In [66]:
len(words), len(dictionary)

(271, 142)

In [67]:
dictionary

array(['But', 'Four', 'God', 'It', 'Liberty', 'Now', 'The', 'We', 'a',
       'above', 'add', 'advanced', 'ago', 'all', 'altogether', 'and',
       'any', 'are', 'as', 'battle-field', 'be', 'before', 'birth',
       'brave', 'brought', 'but', 'by', 'can', 'cause', 'civil', 'come',
       'conceived', 'consecrate', 'consecrated', 'continent', 'created',
       'dead', 'dedicate', 'dedicated', 'detract', 'devotion', 'did',
       'died', 'do', 'earth', 'endure', 'engaged', 'equal', 'far',
       'fathers', 'field', 'final', 'fitting', 'for', 'forget', 'forth',
       'fought', 'freedom', 'from', 'full', 'gave', 'government', 'great',
       'ground', 'hallow', 'have', 'here', 'highly', 'honored', 'in',
       'increased', 'is', 'it', 'larger', 'last', 'little', 'live',
       'lives', 'living', 'long', 'measure', 'men', 'met', 'might',
       'nation', 'never', 'new', 'nobly', 'nor', 'not', 'note', 'of',
       'on', 'or', 'our', 'people', 'perish', 'place', 'poor', 'portion',
       'po

In [68]:
integers

array([  1, 109,  15, 111, 141,  12,  94,  49,  24,  55,  92, 124,  34,
         8,  86,  84,  31,  69,   4,  15,  38, 127, 120, 102, 119,  13,
        81,  17,  35,  47,   5, 133,  17,  46,  69,   8,  62,  29, 132,
       118, 135, 119,  84,  93,  16,  84, 114,  31,  15, 114,  38,  27,
        79,  45,   7,  17,  82,  92,   8,  62,  19,  91, 119, 132,   7,
        65,  30, 127,  37,   8,  99,  91, 119,  50,  18,   8,  51, 107,
        97,  53, 125, 137,  66,  60, 121,  77, 119, 119,  84,  83,  76,
         3,  71,  14,  52,  15, 101, 119, 133, 113,  43, 124,   0,  69,
         8,  73, 110, 133,  27,  89,  37, 133,  27,  89,  32, 133,  27,
        89,  64, 124,  63,   6,  23,  81,  78,  15,  36, 137, 115,  66,
        65,  33,  72,  48,   9,  94,  98, 100, 127,  10,  93,  39,   6,
       140, 138,  75,  90,  88,  79, 105, 134, 133, 108,  66,  25,  72,
        27,  85,  54, 134, 123,  41,  66,   3,  71,  53, 130, 120,  78,
       103, 127,  20,  38,  66, 127, 120, 129, 139, 136, 123, 13

In [69]:
dictionary[integers]

array(['Four', 'score', 'and', 'seven', 'years', 'ago', 'our', 'fathers',
       'brought', 'forth', 'on', 'this', 'continent', 'a', 'new',
       'nation', 'conceived', 'in', 'Liberty', 'and', 'dedicated', 'to',
       'the', 'proposition', 'that', 'all', 'men', 'are', 'created',
       'equal', 'Now', 'we', 'are', 'engaged', 'in', 'a', 'great',
       'civil', 'war', 'testing', 'whether', 'that', 'nation', 'or',
       'any', 'nation', 'so', 'conceived', 'and', 'so', 'dedicated',
       'can', 'long', 'endure', 'We', 'are', 'met', 'on', 'a', 'great',
       'battle-field', 'of', 'that', 'war', 'We', 'have', 'come', 'to',
       'dedicate', 'a', 'portion', 'of', 'that', 'field', 'as', 'a',
       'final', 'resting', 'place', 'for', 'those', 'who', 'here', 'gave',
       'their', 'lives', 'that', 'that', 'nation', 'might', 'live', 'It',
       'is', 'altogether', 'fitting', 'and', 'proper', 'that', 'we',
       'should', 'do', 'this', 'But', 'in', 'a', 'larger', 'sense', 'we',
       '

Notice that slicing with integer indexes is a function in the mathematical sense. An array is a mapping from integers to the values of the array: `[0, N) → V`.

Slicing with integer indexes composes a function `[0, m) → [0, N)` with it to get a new function `[0, m) → V`.

In [70]:
a = numpy.array(["zero", "one", "two", "three", "four"])
b = numpy.array([3, 3, 1, 2, 4, 0, 1])

a[b]

array(['three', 'three', 'one', 'two', 'four', 'zero', 'one'], dtype='<U5')

Putting things together: we can use reshaping and fancy indexing together to do some surprisingly powerful things. ("Look, ma! No for loops!")

For example, I once had to reverse a list in groups of 8. I struggled with it until I found this online:

In [78]:
original = numpy.array([7, 6, 5, 4, 3, 2, 1, 0,
                        15, 14, 13, 12, 11, 10, 9, 8,
                        23, 22, 21, 20, 19, 18, 17, 16,
                        31, 30, 29, 28, 27, 26, 25, 24])

In [82]:
original.reshape(4, 8)[:, ::-1].reshape(32)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

**Question:** How would we change this if we wanted to turn

In [220]:
original = numpy.array([7, 6, 5, 4, 3, 2, 1, 0,
                        15, 14, 13, 12, 11, 10, 9, 8,
                        23, 22, 21, 20, 19, 18, 17, 16,
                        31, 30, 29, 28, 27, 26, 25, 24])

into [31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]?

In [221]:
original.reshape(4, 8)[:, ::-1].reshape(32)    # change this line!

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

Also unlike Python slices, you can even _assign_ to sliced Numpy arrays. This overwrites parts of the array.

In [259]:
array = numpy.arange(10) * 1.1
array

array([0. , 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])

In [262]:
array[5:] = -9
array

array([ 0. ,  1.1,  2.2,  3.3,  4.4, -9. , -9. , -9. , -9. , -9. ])

In [263]:
array[::2] = 123
array

array([123. ,   1.1, 123. ,   3.3, 123. ,  -9. , 123. ,  -9. , 123. ,
        -9. ])

In [265]:
array = numpy.ones(10, dtype=numpy.int64) * 999
array[numpy.array([False, False, False, True, True, False, True, False, True, False])] = numpy.array([1, 2, 3, 4])
array

array([999, 999, 999,   1,   2, 999,   3, 999,   4, 999])

In [266]:
array = numpy.ones(10, dtype=numpy.int64) * 999
array[numpy.array([8, 7, 3, 0])] = numpy.array([1, 2, 3, 4])
array

array([  4, 999, 999,   3, 999, 999, 999,   2,   1, 999])

**Example:** Rubik's square. Let's use the `roll` function to rotate rows and columns of a square matrix (bigger than 3×3).

In [304]:
rubiks = (numpy.tile(numpy.arange(10) * 0.1, 10) + numpy.repeat(numpy.arange(10), 10)).reshape(10, 10)
rubiks

array([[0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
       [1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9],
       [2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9],
       [3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9],
       [4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9],
       [5. , 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9],
       [6. , 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9],
       [7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9],
       [8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9],
       [9. , 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9]])

In [305]:
def twist(isrow, index, howmuch):
    if isrow:
        rubiks[index, :] = numpy.roll(rubiks[index, :], howmuch)
    else:
        rubiks[:, index] = numpy.roll(rubiks[:, index], howmuch)
    return rubiks

In [306]:
twist(True, 3, 5)

array([[0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
       [1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9],
       [2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9],
       [3.5, 3.6, 3.7, 3.8, 3.9, 3. , 3.1, 3.2, 3.3, 3.4],
       [4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9],
       [5. , 5.1, 5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9],
       [6. , 6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9],
       [7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9],
       [8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9],
       [9. , 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9]])

**Exercise:** Shuffling cards. Suppose you want to model the first cut of a deck of cards. The dealer roughly cuts the deck in two and flips them together such that all cards from the left hand are in order relative to each other and all cards from the right hand are in order relative to each other, but they are randomly interleaved.

<center><img src="img/cards-chance-deck-19060.jpg" width="25%" /></center>

In [253]:
cards = numpy.arange(52)
random_booleans = numpy.random.randint(0, 2, 52, dtype=numpy.bool)
negated_booleans = ~random_booleans
solution = cards.copy()    # overwrite this array with your solution!

In [258]:
cut = random_booleans.sum()
left = cards[:cut]
right = cards[cut:]
solution[random_booleans] = left
solution[negated_booleans] = right
solution

array([ 0,  1,  2, 23,  3, 24, 25,  4,  5, 26, 27, 28, 29, 30, 31, 32,  6,
       33,  7,  8, 34,  9, 10, 35, 36, 37, 38, 39, 40, 11, 41, 12, 13, 14,
       42, 15, 43, 44, 45, 46, 16, 47, 17, 18, 19, 20, 48, 49, 50, 51, 21,
       22])